In [16]:
import matplotlib.pyplot as plt


In [4]:
import os
import numpy as np
from PIL import Image
import json


location = "/Users/marcus/Desktop/Training_Data_SVM/Crops_From_4096_2048/"


imageList = []
labelList = []
ytrain = []

for dirName, x, fileList in os.walk(location):
    
    for filename in fileList:
        if(filename[-4:] == ".jpg" and not filename[-7:] == "ged.jpg"):
            
            json_file_location = os.path.join(dirName, filename[:-4] + ".json")
            
            if os.path.exists(json_file_location):
                
                jpg_file_location = os.path.join(dirName, filename)
                imageList.append(str(jpg_file_location))
                labelList.append(str(json_file_location))

for json_file in labelList:
    file = open(json_file)
    data = json.load(file)
    ytrain.append(data["Pano_Data"][0]["label_type"])

    
    
X = np.array([np.array(Image.open(fname)) for fname in imageList])    
y = np.array(ytrain)




In [45]:
def get_sift_dict(image_list, n_classes, sift):
    sift_descriptors = []
    
    
    for image_path in image_list:
        img = cv2.imread(image_path)
        kp, des = sift.detectAndCompute(img, None)
        for d in des:
            sift_descriptors.append(d)
    k = 10 * n_classes
    batch_size = np.size(len(image_list)) * 3
    
    kmeans = MiniBatchKMeans(n_clusters = k, batch_size = batch_size, verbose = 1).fit(sift_descriptors)
    kmeans.verbose = False
    
    return kmeans

def get_sift_features(image_list, feature_dict, sift):
    histo_list = []

    for img_path in image_list:
        img = cv2.imread(img_path)
        kp, des = sift.detectAndCompute(img, None)

        histo = np.zeros(k)
        nkp = np.size(kp)

        for d in des:
            idx = kmeans.predict([d])
            histo[idx] += 1/nkp

        histo_list.append(histo)
    return np.array(histo_list)

In [40]:
sift = cv2.xfeatures2d.SIFT_create()

sift_dict = get_sift_dict(imageList, 4, sift)




Init 1/3 with method: k-means++
Inertia for init 1/3: 331086.000000
Init 2/3 with method: k-means++
Inertia for init 2/3: 0.000000
Init 3/3 with method: k-means++
Inertia for init 3/3: 135676.500000
Minibatch iteration 1/3015400: mean batch inertia: 183261.666667, ewa inertia: 183261.666667 
Minibatch iteration 2/3015400: mean batch inertia: 118373.500000, ewa inertia: 183257.362834 
Minibatch iteration 3/3015400: mean batch inertia: 148272.185185, ewa inertia: 183255.042375 
Minibatch iteration 4/3015400: mean batch inertia: 144410.625000, ewa inertia: 183252.465944 
Minibatch iteration 5/3015400: mean batch inertia: 136444.546667, ewa inertia: 183249.361319 
Minibatch iteration 6/3015400: mean batch inertia: 130645.166667, ewa inertia: 183245.872244 
Minibatch iteration 7/3015400: mean batch inertia: 147757.000000, ewa inertia: 183243.518376 
Minibatch iteration 8/3015400: mean batch inertia: 127984.095238, ewa inertia: 183239.853189 
Minibatch iteration 9/3015400: mean batch inertia

/Users/marcus/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=9 should be larger than k=40. Setting it to 3*k
  init_size=init_size)


Minibatch iteration 253/3015400: mean batch inertia: 90389.899423, ewa inertia: 181888.392731 
Minibatch iteration 254/3015400: mean batch inertia: 53788.149007, ewa inertia: 181879.896236 
Minibatch iteration 255/3015400: mean batch inertia: 79935.485780, ewa inertia: 181873.134576 
Minibatch iteration 256/3015400: mean batch inertia: 69865.154102, ewa inertia: 181865.705431 
Minibatch iteration 257/3015400: mean batch inertia: 108704.868388, ewa inertia: 181860.852898 
Minibatch iteration 258/3015400: mean batch inertia: 110642.851629, ewa inertia: 181856.129227 
Minibatch iteration 259/3015400: mean batch inertia: 92602.213933, ewa inertia: 181850.209289 
[MiniBatchKMeans] Reassigning 1 cluster centers.
Minibatch iteration 260/3015400: mean batch inertia: 91690.324398, ewa inertia: 181844.229261 
Minibatch iteration 261/3015400: mean batch inertia: 85142.160297, ewa inertia: 181837.815310 
Minibatch iteration 262/3015400: mean batch inertia: 82092.471666, ewa inertia: 181831.199508 

Minibatch iteration 457/3015400: mean batch inertia: 84096.575654, ewa inertia: 180654.764418 
Minibatch iteration 458/3015400: mean batch inertia: 92228.971233, ewa inertia: 180648.899407 
Minibatch iteration 459/3015400: mean batch inertia: 113255.284302, ewa inertia: 180644.429396 
Minibatch iteration 460/3015400: mean batch inertia: 98638.911152, ewa inertia: 180638.990222 
Minibatch iteration 461/3015400: mean batch inertia: 83844.218127, ewa inertia: 180632.570122 
[MiniBatchKMeans] Reassigning 1 cluster centers.
Minibatch iteration 462/3015400: mean batch inertia: 44555.672483, ewa inertia: 180623.544560 
Minibatch iteration 463/3015400: mean batch inertia: 85224.945733, ewa inertia: 180617.217064 
Minibatch iteration 464/3015400: mean batch inertia: 105970.110144, ewa inertia: 180612.265951 
Minibatch iteration 465/3015400: mean batch inertia: 96306.730037, ewa inertia: 180606.674224 
Minibatch iteration 466/3015400: mean batch inertia: 97154.042549, ewa inertia: 180601.139068 

Minibatch iteration 746/3015400: mean batch inertia: 79032.259316, ewa inertia: 178896.247204 
Minibatch iteration 747/3015400: mean batch inertia: 43956.141354, ewa inertia: 178887.297041 
[MiniBatchKMeans] Reassigning 1 cluster centers.
Minibatch iteration 748/3015400: mean batch inertia: 98041.457734, ewa inertia: 178881.934785 
Minibatch iteration 749/3015400: mean batch inertia: 108620.428808, ewa inertia: 178877.274556 
Minibatch iteration 750/3015400: mean batch inertia: 49888.542280, ewa inertia: 178868.719130 
Minibatch iteration 751/3015400: mean batch inertia: 105591.416169, ewa inertia: 178863.858871 
Minibatch iteration 752/3015400: mean batch inertia: 111749.822190, ewa inertia: 178859.407404 
Minibatch iteration 753/3015400: mean batch inertia: 110700.504461, ewa inertia: 178854.886633 
Minibatch iteration 754/3015400: mean batch inertia: 63081.976268, ewa inertia: 178847.207772 
Minibatch iteration 755/3015400: mean batch inertia: 100898.312674, ewa inertia: 178842.0376

...

Minibatch iteration 25677/3015400: mean batch inertia: 113669.249662, ewa inertia: 99691.986386 
Minibatch iteration 25678/3015400: mean batch inertia: 61871.188548, ewa inertia: 99689.477849 
Minibatch iteration 25679/3015400: mean batch inertia: 77815.617127, ewa inertia: 99688.027023 
Minibatch iteration 25680/3015400: mean batch inertia: 57293.284041, ewa inertia: 99685.215110 
Minibatch iteration 25681/3015400: mean batch inertia: 105711.650621, ewa inertia: 99685.614825 
Minibatch iteration 25682/3015400: mean batch inertia: 55690.899144, ewa inertia: 99682.696791 
Minibatch iteration 25683/3015400: mean batch inertia: 73447.036041, ewa inertia: 99680.956660 
Minibatch iteration 25684/3015400: mean batch inertia: 96564.766783, ewa inertia: 99680.749973 
Minibatch iteration 25685/3015400: mean batch inertia: 46676.089455, ewa inertia: 99677.234337 
Minibatch iteration 25686/3015400: mean batch inertia: 79346.395525, ewa inertia: 99675.885855 
Minibatch iteration 25687/3015400: mea

Minibatch iteration 25965/3015400: mean batch inertia: 120608.873827, ewa inertia: 99355.847370 
Minibatch iteration 25966/3015400: mean batch inertia: 100416.907115, ewa inertia: 99355.917746 
Minibatch iteration 25967/3015400: mean batch inertia: 103543.008157, ewa inertia: 99356.195463 
Minibatch iteration 25968/3015400: mean batch inertia: 72969.409845, ewa inertia: 99354.445309 
Minibatch iteration 25969/3015400: mean batch inertia: 85192.618134, ewa inertia: 99353.505998 
Minibatch iteration 25970/3015400: mean batch inertia: 42532.916162, ewa inertia: 99349.737263 
Minibatch iteration 25971/3015400: mean batch inertia: 75082.697358, ewa inertia: 99348.127705 
Minibatch iteration 25972/3015400: mean batch inertia: 56666.840256, ewa inertia: 99345.296787 
Minibatch iteration 25973/3015400: mean batch inertia: 75191.920351, ewa inertia: 99343.694768 
Minibatch iteration 25974/3015400: mean batch inertia: 61247.092395, ewa inertia: 99341.167937 
Minibatch iteration 25975/3015400: me

Minibatch iteration 26220/3015400: mean batch inertia: 78410.530357, ewa inertia: 99045.034554 
Minibatch iteration 26221/3015400: mean batch inertia: 80573.151197, ewa inertia: 99043.809371 
Minibatch iteration 26222/3015400: mean batch inertia: 102383.286167, ewa inertia: 99044.030868 
Minibatch iteration 26223/3015400: mean batch inertia: 94355.295635, ewa inertia: 99043.719878 
Minibatch iteration 26224/3015400: mean batch inertia: 72236.362921, ewa inertia: 99041.941829 
Minibatch iteration 26225/3015400: mean batch inertia: 64583.835634, ewa inertia: 99039.656329 
Minibatch iteration 26226/3015400: mean batch inertia: 73737.754111, ewa inertia: 99037.978131 
Minibatch iteration 26227/3015400: mean batch inertia: 87514.420471, ewa inertia: 99037.213809 
Minibatch iteration 26228/3015400: mean batch inertia: 48152.731142, ewa inertia: 99033.838798 
Minibatch iteration 26229/3015400: mean batch inertia: 81271.846613, ewa inertia: 99032.660699 
Minibatch iteration 26230/3015400: mean

Minibatch iteration 26439/3015400: mean batch inertia: 47464.536762, ewa inertia: 98752.101066 
Minibatch iteration 26440/3015400: mean batch inertia: 80324.338568, ewa inertia: 98750.878809 
Minibatch iteration 26441/3015400: mean batch inertia: 65786.292747, ewa inertia: 98748.692370 
Minibatch iteration 26442/3015400: mean batch inertia: 77987.604154, ewa inertia: 98747.315351 
Minibatch iteration 26443/3015400: mean batch inertia: 133112.286154, ewa inertia: 98749.594674 
Minibatch iteration 26444/3015400: mean batch inertia: 94052.239224, ewa inertia: 98749.283112 
Minibatch iteration 26445/3015400: mean batch inertia: 44598.714818, ewa inertia: 98745.691472 
Minibatch iteration 26446/3015400: mean batch inertia: 110960.222057, ewa inertia: 98746.501624 
Minibatch iteration 26447/3015400: mean batch inertia: 103358.437707, ewa inertia: 98746.807519 
Minibatch iteration 26448/3015400: mean batch inertia: 81201.230280, ewa inertia: 98745.643775 
Minibatch iteration 26449/3015400: me

Minibatch iteration 26659/3015400: mean batch inertia: 79043.387024, ewa inertia: 98449.461694 
Minibatch iteration 26660/3015400: mean batch inertia: 71060.457529, ewa inertia: 98447.645066 
Minibatch iteration 26661/3015400: mean batch inertia: 87085.190341, ewa inertia: 98446.891429 
Minibatch iteration 26662/3015400: mean batch inertia: 74968.636767, ewa inertia: 98445.334188 
Minibatch iteration 26663/3015400: mean batch inertia: 90102.965936, ewa inertia: 98444.780865 
Minibatch iteration 26664/3015400: mean batch inertia: 100630.036360, ewa inertia: 98444.925806 
Minibatch iteration 26665/3015400: mean batch inertia: 99470.264545, ewa inertia: 98444.993814 
Minibatch iteration 26666/3015400: mean batch inertia: 52205.904943, ewa inertia: 98441.926917 
Minibatch iteration 26667/3015400: mean batch inertia: 75115.928819, ewa inertia: 98440.379775 
Minibatch iteration 26668/3015400: mean batch inertia: 101935.873419, ewa inertia: 98440.611621 
Minibatch iteration 26669/3015400: mea

In [46]:
sift_features = get_sift_features(imageList, sift_dict, sift)

In [47]:
sift_features.shape


(295, 40)

In [48]:
from sklearn.utils import shuffle

X = sift_features
y = y.reshape(y.shape[0],)

X, y = shuffle(X, y, random_state=82)

In [49]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(test, y, test_size = 0.5, random_state = 82)

clf = svm.SVC()

clf.fit(X_train, y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [53]:
from sklearn.metrics import accuracy_score


sklearn.metrics.confusion_matrix(y_test, preds)
preds = clf.predict(X_train)
print("Accurancy:", accuracy_score(y_train, preds))

Accurancy: 0.6462585034013606


In [36]:
testImage = np.array(np.array(Image.open("/Users/marcus/Desktop/test.jpeg")))
testImage = testImage.reshape(testImage[1]*testImage[2], testImage[0]).T




print(clf.predict(X_test[:3]))
plt.imshow(X_test)


TypeError: only integer scalar arrays can be converted to a scalar index

In [54]:
import sklearn
sklearn.metrics.confusion_matrix(y_train, preds)

array([[95,  0,  0,  0],
       [12,  0,  0,  0],
       [17,  0,  0,  0],
       [23,  0,  0,  0]])

In [17]:
lmao = np.array(sift_descriptors)

In [18]:
lmao.shape

(295,)

In [20]:
lmao[6].shape

(430, 128)

In [7]:
sift_descriptors = np.array(sift_descriptors)
for n in sift_descriptors:
    print(n.shape)


(10, 128)
(10, 128)
(10, 128)
(12, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(11, 128)
(11, 128)
(11, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(11, 128)
(11, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(12, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(11, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(12, 128)
(11, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)
(10, 128)
(11, 128)
(10, 128)
(10, 128)


In [11]:
from sklearn.cluster import KMeans

stuff = []

for x in range(len(sift_descriptors)):
    stuff.append([sift_descriptors[x]])

[[array([[  0.,   0.,   0., ...,   4.,   3.,  14.],
         [ 61.,  44.,   0., ...,   0.,   0.,   0.],
         [  2.,   0.,   0., ...,   0.,   0.,   1.],
         ...,
         [126., 110.,  29., ...,   0.,   0.,   0.],
         [  9.,  17., 105., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   1.]], dtype=float32)],
 [array([[ 96.,  92.,   2., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,  49., 155.,  22.],
         [  1.,   6.,  37., ...,   0.,   0.,  18.],
         ...,
         [  0.,   1.,   1., ...,   1.,   0.,   0.],
         [  1.,   0.,   0., ...,   6.,   1.,   4.],
         [ 23.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)],
 [array([[  0.,   0.,   0., ...,   2.,   1.,  12.],
         [  0.,   0.,   0., ..., 101., 129.,  13.],
         [  5.,  22.,  32., ...,   2.,   1.,   3.],
         ...,
         [  0.,   0.,   1., ...,   0.,   5.,  22.],
         [  0.,   0.,   7., ...,  36.,   6.,  13.],
         [  3.,   3.,   1., ...,   4.,